# Predicting Graduation Rates Data Wrangling

In [1]:
import pandas as pd
import numpy as np
import sys
from functools import reduce
pd.options.display.max_columns = 200
pd.options.display.max_rows = 170

## Viewing the dataframe

In [2]:
df = pd.read_csv("data/UniversityData.csv", index_col="UnitID")

In [3]:
df.drop("Unnamed: 136", axis=1, inplace=True)

In [4]:
schema = pd.read_csv("data/Schema.csv")

In [5]:
df.head()

,Institution Name,City location of institution (HD2019),Grand total (S2019_IS All instructional staff),Grand total men (S2019_IS All instructional staff),Grand total women (S2019_IS All instructional staff),American Indian or Alaska Native total (S2019_IS All instructional staff),American Indian or Alaska Native men (S2019_IS All instructional staff),American Indian or Alaska Native women (S2019_IS All instructional staff),Asian total (S2019_IS All instructional staff),Asian men (S2019_IS All instructional staff),Asian women (S2019_IS All instructional staff),Black or African American total (S2019_IS All instructional staff),Black or African American men (S2019_IS All instructional staff),Black or African American women (S2019_IS All instructional staff),Hispanic or Latino total (S2019_IS All instructional staff),Hispanic or Latino men (S2019_IS All instructional staff),Hispanic or Latino women (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander total (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander men (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander women (S2019_IS All instructional staff),White total (S2019_IS All instructional staff),White men (S2019_IS All instructional staff),White women (S2019_IS All instructional staff),Two or more races total (S2019_IS All instructional staff),Two or more races men (S2019_IS All instructional staff),Two or more races women (S2019_IS All instructional staff),Race/ethnicity unknown total (S2019_IS All instructional staff),Race/ethnicity unknown men (S2019_IS All instructional staff),Race/ethnicity unknown women (S2019_IS All instructional staff),Nonresident alien total (S2019_IS All instructional staff),Nonresident alien men (S2019_IS All instructional staff),Nonresident alien women (S2019_IS All instructional staff),Graduation rate total cohort (DRVGR2019),Graduation rate men (DRVGR2019),Graduation rate women (DRVGR2019),Graduation rate American Indian or Alaska Native (DRVGR2019),Graduation rate Asian/Native Hawaiian/Other Pacific Islander (DRVGR2019),Graduation rate Asian (DRVGR2019),Graduation rate Native Hawaiian or Other Pacific Islander (DRVGR2019),Graduation rate Black non-Hispanic (DRVGR2019),Graduation rate Hispanic (DRVGR2019),Graduation rate White non-Hispanic (DRVGR2019),Graduation rate two or more races (DRVGR2019),Graduation rate Race/ethnicity unknown (DRVGR2019),Graduation rate Nonresident alien (DRVGR2019),Transfer-out rate total cohort (DRVGR2019),Percent of full-time first-time undergraduates awarded any financial aid (SFA1819),Percent of full-time first-time undergraduates awarded federal state local or institutional grant aid (SFA1819),Core revenues total dollars (GASB) (DRVF2019),Tuition and fees as a percent of core revenues (GASB) (DRVF2019),State appropriations as percent of core revenues (GASB) (DRVF2019),Local appropriations as a percent of core revenues (GASB) (DRVF2019),Government grants and contracts as a percent of core revenues (GASB) (DRVF2019),Private gifts grants and contracts as a percent of core revenues (GASB) (DRVF2019),Investment return as a percent of core revenues (GASB) (DRVF2019),Other revenues as a percent of core revenues (GASB) (DRVF2019),Core revenues total dollars (FASB) (DRVF2019),Tuition and fees as a percent of core revenues (FASB) (DRVF2019),Government grants and contracts as a percent of core revenues (FASB) (DRVF2019),Private gifts grants contracts/contributions from affiliated entities as a percent of core revenues (FASB) (DRVF2019),Investment return as a percent of core revenues (FASB) (DRVF2019),Other revenues as a percent of core revenues (FASB) (DRVF2019),Core revenues total dollars (for-profit institutions) (DRVF2019),Tuition and fees as a percent of core revenues (for-profit institutions) (DRVF2019),Government grants and contracts as a percent of core revenues (for-profit institutions) (DRVF2019),Private gifts grants contracts as a percent of co

In [6]:
df.columns[32]

'Graduation rate  total cohort (DRVGR2019)'

In [7]:
#drop where total graduation rate is missing
df.dropna(subset=[df.columns[32]], inplace=True)

In [8]:
df.shape

(2315, 135)

In [9]:
schema[schema["VariableName"].str.contains("Parent")]

,VariableName,Value,ValueLabel
135,Parent/child indicator - Finance (FLAGS2019),1,Parent record - includes data from branch camp...
136,Parent/child indicator - Finance (FLAGS2019),2,Child record - data reported with parent campus
137,Parent/child indicator - Finance (FLAGS2019),3,Partial child record - reports revenues/expens...
138,Parent/child indicator - Finance (FLAGS2019),5,Child record - reports partial data but other ...
139,Parent/child indicator - Finance (FLAGS2019),-2,Not applicable


## Spliting data frame into dependent variables, categorical and ordinal features, and numerical features

In [10]:
#list of categorical features
cat_ordinal_groups = list(schema.groupby("VariableName").groups.keys())

In [11]:
categorical_df = df.loc[:,cat_ordinal_groups] #dataframe with categorical features

In [12]:
df.columns[32:46]

Index(['Graduation rate  total cohort (DRVGR2019)',
       'Graduation rate  men (DRVGR2019)',
       'Graduation rate  women (DRVGR2019)',
       'Graduation rate  American Indian or Alaska Native (DRVGR2019)',
       'Graduation rate  Asian/Native Hawaiian/Other Pacific Islander (DRVGR2019)',
       'Graduation rate  Asian (DRVGR2019)',
       'Graduation rate  Native Hawaiian or Other Pacific Islander (DRVGR2019)',
       'Graduation rate  Black  non-Hispanic (DRVGR2019)',
       'Graduation rate  Hispanic (DRVGR2019)',
       'Graduation rate  White  non-Hispanic (DRVGR2019)',
       'Graduation rate  two or more races (DRVGR2019)',
       'Graduation rate  Race/ethnicity unknown (DRVGR2019)',
       'Graduation rate  Nonresident alien (DRVGR2019)',
       'Transfer-out rate  total cohort (DRVGR2019)'],
      dtype='object')

In [13]:
#dependent features list
dependent_var_list = list(df.columns[32:46])

In [14]:
dependent_df = df.loc[:,dependent_var_list]

In [15]:
#Numeric Data Frame will be all of those columns outside of dependent and categorical/ordinal

In [16]:
not_numeric = cat_ordinal_groups + dependent_var_list

In [17]:
numeric_df = df.drop(not_numeric, axis=1)

# Viewing dataframe by dtypes

In [18]:
numeric_df.describe()

,Grand total (S2019_IS All instructional staff),Grand total men (S2019_IS All instructional staff),Grand total women (S2019_IS All instructional staff),American Indian or Alaska Native total (S2019_IS All instructional staff),American Indian or Alaska Native men (S2019_IS All instructional staff),American Indian or Alaska Native women (S2019_IS All instructional staff),Asian total (S2019_IS All instructional staff),Asian men (S2019_IS All instructional staff),Asian women (S2019_IS All instructional staff),Black or African American total (S2019_IS All instructional staff),Black or African American men (S2019_IS All instructional staff),Black or African American women (S2019_IS All instructional staff),Hispanic or Latino total (S2019_IS All instructional staff),Hispanic or Latino men (S2019_IS All instructional staff),Hispanic or Latino women (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander total (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander men (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander women (S2019_IS All instructional staff),White total (S2019_IS All instructional staff),White men (S2019_IS All instructional staff),White women (S2019_IS All instructional staff),Two or more races total (S2019_IS All instructional staff),Two or more races men (S2019_IS All instructional staff),Two or more races women (S2019_IS All instructional staff),Race/ethnicity unknown total (S2019_IS All instructional staff),Race/ethnicity unknown men (S2019_IS All instructional staff),Race/ethnicity unknown women (S2019_IS All instructional staff),Nonresident alien total (S2019_IS All instructional staff),Nonresident alien men (S2019_IS All instructional staff),Nonresident alien women (S2019_IS All instructional staff),Percent of full-time first-time undergraduates awarded any financial aid (SFA1819),Percent of full-time first-time undergraduates awarded federal state local or institutional grant aid (SFA1819),Core revenues total dollars (GASB) (DRVF2019),Tuition and fees as a percent of core revenues (GASB) (DRVF2019),State appropriations as percent of core revenues (GASB) (DRVF2019),Local appropriations as a percent of core revenues (GASB) (DRVF2019),Government grants and contracts as a percent of core revenues (GASB) (DRVF2019),Private gifts grants and contracts as a percent of core revenues (GASB) (DRVF2019),Investment return as a percent of core revenues (GASB) (DRVF2019),Other revenues as a percent of core revenues (GASB) (DRVF2019),Core revenues total dollars (FASB) (DRVF2019),Tuition and fees as a percent of core revenues (FASB) (DRVF2019),Government grants and contracts as a percent of core revenues (FASB) (DRVF2019),Private gifts grants contracts/contributions from affiliated entities as a percent of core revenues (FASB) (DRVF2019),Investment return as a percent of core revenues (FASB) (DRVF2019),Other revenues as a percent of core revenues (FASB) (DRVF2019),Core revenues total dollars (for-profit institutions) (DRVF2019),Tuition and fees as a percent of core revenues (for-profit institutions) (DRVF2019),Government grants and contracts as a percent of core revenues (for-profit institutions) (DRVF2019),Private gifts grants contracts as a percent of core revenues (for-profit institutions) (DRVF2019),Investment return as a percent of core revenues (for-profit institutions) (DRVF2019),Sales and services of educational activities as a percent of core revenues (for-profit institutions) (DRVF2019),Other revenues as a percent of core revenues (for-profit institutions) (DRVF2019),Core expenses total dollars (GASB) (DRVF2019),Instruction expenses as a percent of total core expenses (GASB) (DRVF2019),Research expenses as a percent of total core expenses (GASB) (DRVF2019),Public service expenses as a percent of total core expenses (GASB) (DRVF2019),Academic support expenses as a percent of total core expenses (GASB) (DRVF2019),Student service expenses as a percent 

In [19]:
dependent_df.reset_index(inplace=True)

In [20]:
dependent_df.head()

,UnitID,Graduation rate total cohort (DRVGR2019),Graduation rate men (DRVGR2019),Graduation rate women (DRVGR2019),Graduation rate American Indian or Alaska Native (DRVGR2019),Graduation rate Asian/Native Hawaiian/Other Pacific Islander (DRVGR2019),Graduation rate Asian (DRVGR2019),Graduation rate Native Hawaiian or Other Pacific Islander (DRVGR2019),Graduation rate Black non-Hispanic (DRVGR2019),Graduation rate Hispanic (DRVGR2019),Graduation rate White non-Hispanic (DRVGR2019),Graduation rate two or more races (DRVGR2019),Graduation rate Race/ethnicity unknown (DRVGR2019),Graduation rate Nonresident alien (DRVGR2019),Transfer-out rate total cohort (DRVGR2019)
0,180203,29.0,13.0,46.0,26.0,NaN,NaN,NaN,NaN,NaN,33.0,NaN,NaN,NaN,NaN
1,222178,61.0,56.0,65.0,25.0,58.0,58.0,NaN,53.0,50.0,67.0,54.0,0.0,57.0,NaN
2,138558,26.0,28.0,25.0,33.0,25.0,25.0,NaN,12.0,32.0,29.0,20.0,100.0,33.0,39.0
3,172866,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
4,108232,45.0,38.0,51.0,67.0,56.0,57.0,40.0,20.0,33.0,50.0,40.0,28.0,57.0,14.0


In [21]:
graduation_df = dependent_df.iloc[:,:2]

In [22]:
graduation_df

,UnitID,Graduation rate total cohort (DRVGR2019)
0,180203,29.0
1,222178,61.0
2,138558,26.0
3,172866,0.0
4,108232,45.0
...,...,...
2310,217059,63.0
2311,141361,42.0
2312,206695,41.0
2313,204255,54.0


In [23]:
categorical_df

,Carnegie Classification 2018: Basic (HD2018),Carnegie Classification 2018: Enrollment Profile (HD2018),Carnegie Classification 2018: Size and Setting (HD2018),Historically Black College or University (HD2018),Institution size category (HD2018),Parent/child indicator - Finance (FLAGS2019),Sector of institution (HD2018),State abbreviation (HD2018)
UnitID,,,,,,,,
180203,33.0,1.0,1.0,2.0,1.0,-2.0,4.0,MT
222178,18.0,4.0,14.0,2.0,3.0,-2.0,2.0,TX
138558,23.0,2.0,10.0,2.0,2.0,-2.0,1.0,GA
172866,28.0,2.0,6.0,2.0,1.0,-2.0,3.0,MN
108232,18.0,5.0,12.0,2.0,4.0,-2.0,3.0,CA
...,...,...,...,...,...,...,...,...
217059,20.0,3.0,14.0,2.0,2.0,-2.0,2.0,PA
141361,21.0,2.0,11.0,2.0,2.0,-2.0,2.0,GA
206695,18.0,3.0,15.0,2.0,4.0,-2.0,1.0,OH


# Data Wrangling - Filling Missing Values

### Cleaning Categorical Variables

In [24]:
categorical_df.fillna("isMissing", inplace=True)

In [25]:
for i in categorical_df.columns:
    categorical_df[i] = categorical_df[i].astype(str)

In [26]:
nm = categorical_df.columns

In [27]:
float_to_int = {str(float(i)) : str(i) for i in range(-2, 35)}

In [28]:
for i in nm:
    categorical_df[i].replace(float_to_int, inplace=True)

In [29]:
categorical_df[nm[0]].unique()

array(['33', '18', '23', '28', '26', '17', '22', '21', '30', '19', '24',
       '20', '14', 'isMissing', '29', '-2', '10', '16', '6', '15', '4',
       '5', '7', '2', '13', '32', '8', '1', '9', '27', '11', '25', '12'],
      dtype=object)

# map the actual values of the categories instead of numbers so we don't have to refer back to schema

In [30]:
maps = {}

for i in cat_ordinal_groups:    
    dict_to_add = {}
   
    val_label_df = schema[schema["VariableName"] == i]

    valLabelDf = val_label_df[["Value", "ValueLabel"]]


    val_label_dict = valLabelDf.set_index("Value").transpose().to_dict()

    for j,v in val_label_dict.items():

        val_label_dict[j] = v['ValueLabel']
    
    maps[i] = val_label_dict
    
for i in cat_ordinal_groups:
    categorical_df[i].replace(maps[i], inplace=True)
    

    

In [31]:
categorical_df["City location of institution (HD2019)"] = df[df.columns[1]]

In [32]:
#for time and simplicity sake for our baseline model we will drop city because it has over 1000 unique values
categorical_df.head()

,Carnegie Classification 2018: Basic (HD2018),Carnegie Classification 2018: Enrollment Profile (HD2018),Carnegie Classification 2018: Size and Setting (HD2018),Historically Black College or University (HD2018),Institution size category (HD2018),Parent/child indicator - Finance (FLAGS2019),Sector of institution (HD2018),State abbreviation (HD2018),City location of institution (HD2019)
UnitID,,,,,,,,,
180203,Tribal Colleges,Exclusively undergraduate two-year,"Two-year, very small",No,"Under 1,000",Not applicable,"Public, 2-year",Montana,Harlem
222178,Master's Colleges & Universities: Larger Programs,High undergraduate,"Four-year, medium, highly residential",No,"5,000 - 9,999",Not applicable,"Private not-for-profit, 4-year or above",Texas,Abilene
138558,Baccalaureate/Associate's Colleges: Mixed Bacc...,Exclusively undergraduate four-year,"Four-year, small, primarily residential",No,"1,000 - 4,999",Not applicable,"Public, 4-year or above",Georgia,Tifton
172866,Special Focus Four-Year: Other Technology-Rela...,Exclusively undergraduate four-year,"Four-year, very small, primarily nonresidential",No,"Under 1,000",Not applicable,"Private for-profit, 4-year or above",Minnesota,Bloomington
108232,Master's Colleges & Universities: Larger Programs,Majority undergraduate,"Four-year, medium, primarily nonresidential",No,"10,000 - 19,999",Not applicable,"Private for-profit, 4-year or above",California,San Francisco


In [33]:
categorical_df_no_city = categorical_df.drop(
    categorical_df.columns[-1],
    axis=1
)

# Cleaning for baseline model: Numeric 
## Merging Financial Data

Financial data is separated between three groups: GASB, FASB, and For profit institutions. Merge into mini functions combines this data into one column instead of three separate columns for financial data.

In [35]:
def merge_into_mini(dframe, list_of_cols, final_col_name):
    df_list = []
    for i in list_of_cols:
        df_1 = pd.DataFrame(dframe.copy(deep=True)[i])
        df_1.reset_index(inplace=True)
        df_1.dropna(inplace=True)
        df_1.columns = ["UnitID", final_col_name]
        df_list.append(df_1)
    df_final = pd.concat(df_list)
    return df_final

In [36]:
numeric_df.columns[55:75]

Index(['Core expenses  total dollars (GASB) (DRVF2019)',
       'Instruction expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Research expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Public service expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Academic support expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Student service expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Institutional support expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Other core expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Core expenses  total dollars (FASB) (DRVF2019)',
       'Instruction expenses as a percent of total core expenses (FASB) (DRVF2019)',
       'Research expenses as a percent of total core expenses (FASB) (DRVF2019)',
       'Public service expenses as a percent of total core expenses (FASB) (DRVF2019)',
       'Academic s

In [37]:
#create expenses classes
class Container:
    def __init__(self, col_name):
        self.main_list = []
        self.name = col_name.title().replace(" ", "_")
        for i in numeric_df.columns:
            if col_name in i:
                self.main_list.append(i)
    def __repr__(self):
        return self.name

expenses_list = [
    Container("Core expenses"), Container("Instruction expenses"), 
    Container("Research expenses"), Container("Public service expenses"),
    Container("Academic support expenses"),
    Container("Student service expenses"), Container("Institutional support expenses"),
    Container("Other core expenses")
]

revenues_list = [
    Container("Core revenues"), Container("Tuition and fees"),
    Container("Government grants"), Container("Private gifts"),
    Container("Investment return"), Container("Sales and services"),
    Container("Other revenues")
    
]
                   

In [38]:
expenses_df_list = [merge_into_mini(numeric_df, i.main_list, repr(i)) for i in expenses_list]

In [39]:
expenses_df = reduce(lambda x, y: pd.merge(x, y, on = 'UnitID', how="outer"), expenses_df_list)

In [40]:
revenues_df_list = [merge_into_mini(numeric_df, i.main_list, repr(i)) for i in revenues_list]    

In [41]:
revenues_df = reduce(lambda x, y: pd.merge(x, y, on = 'UnitID', how="outer"), revenues_df_list)

In [42]:
revenues_df.fillna(value=0, inplace=True)

In [43]:
#converting to dollar amounts instead of percentages

for i in revenues_df.columns[2:]:
    col_name = i + "_As_Dollar_Amount"
    revenues_df[col_name] = revenues_df[i].apply(lambda x: x / 100) * revenues_df["Core_Revenues"]




In [44]:
revenues_df.head()

,UnitID,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues,Tuition_And_Fees_As_Dollar_Amount,Government_Grants_As_Dollar_Amount,Private_Gifts_As_Dollar_Amount,Investment_Return_As_Dollar_Amount,Sales_And_Services_As_Dollar_Amount,Other_Revenues_As_Dollar_Amount
0,180203,7078959.0,2.0,60.0,7.0,1.0,0.0,29.0,141579.18,4247375.40,495527.13,70789.59,0.0,2052898.11
1,138558,70491879.0,17.0,17.0,2.0,0.0,0.0,28.0,11983619.43,11983619.43,1409837.58,0.00,0.0,19737726.12
2,126182,52099218.0,34.0,51.0,5.0,1.0,0.0,9.0,17713734.12,26570601.18,2604960.90,520992.18,0.0,4688929.62
3,100654,143112797.0,33.0,30.0,2.0,0.0,0.0,6.0,47227223.01,42933839.10,2862255.94,0.00,0.0,8586767.82
4,100724,134993142.0,30.0,25.0,2.0,1.0,0.0,7.0,40497942.60,33748285.50,2699862.84,1349931.42,0.0,9449519.94


In [45]:
expenses_df.fillna(value=0, inplace=True)

In [46]:
#converting to dollar amounts instead of percentages

for i in expenses_df.columns[2:]:
    col_name = i + "_As_Dollar_Amount"
    expenses_df[col_name] = expenses_df[i].apply(lambda x: x / 100) * expenses_df["Core_Expenses"]



In [47]:
expenses_df.head()

,UnitID,Core_Expenses,Instruction_Expenses,Research_Expenses,Public_Service_Expenses,Academic_Support_Expenses,Student_Service_Expenses,Institutional_Support_Expenses,Other_Core_Expenses,Instruction_Expenses_As_Dollar_Amount,Research_Expenses_As_Dollar_Amount,Public_Service_Expenses_As_Dollar_Amount,Academic_Support_Expenses_As_Dollar_Amount,Student_Service_Expenses_As_Dollar_Amount,Institutional_Support_Expenses_As_Dollar_Amount,Other_Core_Expenses_As_Dollar_Amount
0,180203,7294344.0,29.0,3.0,7.0,3.0,8.0,16.0,36.0,2115359.76,218830.32,510604.08,218830.32,583547.52,1167095.04,2625963.84
1,138558,50201124.0,42.0,0.0,3.0,15.0,9.0,19.0,13.0,21084472.08,0.00,1506033.72,7530168.60,4518101.16,9538213.56,6526146.12
2,126182,35725137.0,53.0,0.0,3.0,7.0,18.0,15.0,5.0,18934322.61,0.00,1071754.11,2500759.59,6430524.66,5358770.55,1786256.85
3,100654,146878227.0,22.0,9.0,9.0,6.0,14.0,11.0,29.0,32313209.94,13219040.43,13219040.43,8812693.62,20562951.78,16156604.97,42594685.83
4,100724,134888910.0,29.0,3.0,4.0,9.0,12.0,27.0,16.0,39117783.90,4046667.30,5395556.40,12140001.90,16186669.20,36420005.70,21582225.60


In [48]:
finances_df = revenues_df.merge(expenses_df, how="outer", on="UnitID")

In [49]:
finances_df.head()

,UnitID,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues,Tuition_And_Fees_As_Dollar_Amount,Government_Grants_As_Dollar_Amount,Private_Gifts_As_Dollar_Amount,Investment_Return_As_Dollar_Amount,Sales_And_Services_As_Dollar_Amount,Other_Revenues_As_Dollar_Amount,Core_Expenses,Instruction_Expenses,Research_Expenses,Public_Service_Expenses,Academic_Support_Expenses,Student_Service_Expenses,Institutional_Support_Expenses,Other_Core_Expenses,Instruction_Expenses_As_Dollar_Amount,Research_Expenses_As_Dollar_Amount,Public_Service_Expenses_As_Dollar_Amount,Academic_Support_Expenses_As_Dollar_Amount,Student_Service_Expenses_As_Dollar_Amount,Institutional_Support_Expenses_As_Dollar_Amount,Other_Core_Expenses_As_Dollar_Amount
0,180203,7078959.0,2.0,60.0,7.0,1.0,0.0,29.0,141579.18,4247375.40,495527.13,70789.59,0.0,2052898.11,7294344.0,29.0,3.0,7.0,3.0,8.0,16.0,36.0,2115359.76,218830.32,510604.08,218830.32,583547.52,1167095.04,2625963.84
1,138558,70491879.0,17.0,17.0,2.0,0.0,0.0,28.0,11983619.43,11983619.43,1409837.58,0.00,0.0,19737726.12,50201124.0,42.0,0.0,3.0,15.0,9.0,19.0,13.0,21084472.08,0.00,1506033.72,7530168.60,4518101.16,9538213.56,6526146.12
2,126182,52099218.0,34.0,51.0,5.0,1.0,0.0,9.0,17713734.12,26570601.18,2604960.90,520992.18,0.0,4688929.62,35725137.0,53.0,0.0,3.0,7.0,18.0,15.0,5.0,18934322.61,0.00,1071754.11,2500759.59,6430524.66,5358770.55,1786256.85
3,100654,143112797.0,33.0,30.0,2.0,0.0,0.0,6.0,47227223.01,42933839.10,2862255.94,0.00,0.0,8586767.82,146878227.0,22.0,9.0,9.0,6.0,14.0,11.0,29.0,32313209.94,13219040.43,13219040.43,8812693.62,20562951.78,16156604.97,42594685.83
4,100724,134993142.0,30.0,25.0,2.0,1.0,0.0,7.0,40497942.60,33748285.50,2699862.84,1349931.42,0.0,9449519.94,134888910.0,29.0,3.0,4.0,9.0,12.0,27.0,16.0,39117783.90,4046667.30,5395556.40,12140001.90,16186669.20,36420005.70,21582225.60


## Staff Diversity

<ol>
    <li>All expressed as percentages</li>
    <li>Split between Black, White, Hispanic/Latino, Asian, Other</li>
</ol>

In [50]:
staff_df = numeric_df.loc[:,numeric_df.columns[2:32]]

In [54]:
staff_df.head()

,Grand total (S2019_IS All instructional staff),Grand total men (S2019_IS All instructional staff),Grand total women (S2019_IS All instructional staff),American Indian or Alaska Native total (S2019_IS All instructional staff),American Indian or Alaska Native men (S2019_IS All instructional staff),American Indian or Alaska Native women (S2019_IS All instructional staff),Asian total (S2019_IS All instructional staff),Asian men (S2019_IS All instructional staff),Asian women (S2019_IS All instructional staff),Black or African American total (S2019_IS All instructional staff),Black or African American men (S2019_IS All instructional staff),Black or African American women (S2019_IS All instructional staff),Hispanic or Latino total (S2019_IS All instructional staff),Hispanic or Latino men (S2019_IS All instructional staff),Hispanic or Latino women (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander total (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander men (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander women (S2019_IS All instructional staff),White total (S2019_IS All instructional staff),White men (S2019_IS All instructional staff),White women (S2019_IS All instructional staff),Two or more races total (S2019_IS All instructional staff),Two or more races men (S2019_IS All instructional staff),Two or more races women (S2019_IS All instructional staff),Race/ethnicity unknown total (S2019_IS All instructional staff),Race/ethnicity unknown men (S2019_IS All instructional staff),Race/ethnicity unknown women (S2019_IS All instructional staff),Nonresident alien total (S2019_IS All instructional staff),Nonresident alien men (S2019_IS All instructional staff),Nonresident alien women (S2019_IS All instructional staff)
UnitID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
180203,13.0,7.0,6.0,8.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
222178,265.0,154.0,111.0,1.0,1.0,0.0,3.0,2.0,1.0,14.0,7.0,7.0,13.0,8.0,5.0,0.0,0.0,0.0,228.0,131.0,97.0,3.0,2.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0
138558,149.0,77.0,72.0,0.0,0.0,0.0,10.0,5.0,5.0,10.0,7.0,3.0,3.0,2.0,1.0,0.0,0.0,0.0,124.0,62.0,62.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
172866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108232,219.0,124.0,95.0,0.0,0.0,0.0,22.0,6.0,16.0,4.0,3.0,1.0,8.0,5.0,3.0,3.0,1.0,2.0,141.0,84.0,57.0,2.0,1.0,1.0,39.0,24.0,15.0,0.0,0.0,0.0


In [55]:
#make column names shorter
staff_df_col_names = []
for i in staff_df.columns:
    if "Unit" not in i: 
        col_name = i.replace(i[-36:], " instructional_staff")
    else:
        col_name = i
    
    staff_df_col_names.append(col_name)

In [56]:
staff_df_col_names

['Grand total instructional_staff',
 'Grand total men instructional_staff',
 'Grand total women instructional_staff',
 'American Indian or Alaska Native total instructional_staff',
 'American Indian or Alaska Native men instructional_staff',
 'American Indian or Alaska Native women instructional_staff',
 'Asian total instructional_staff',
 'Asian men instructional_staff',
 'Asian women instructional_staff',
 'Black or African American total instructional_staff',
 'Black or African American men instructional_staff',
 'Black or African American women instructional_staff',
 'Hispanic or Latino total instructional_staff',
 'Hispanic or Latino men instructional_staff',
 'Hispanic or Latino women instructional_staff',
 'Native Hawaiian or Other Pacific Islander total instructional_staff',
 'Native Hawaiian or Other Pacific Islander men instructional_staff',
 'Native Hawaiian or Other Pacific Islander women instructional_staff',
 'White total instructional_staff',
 'White men instructional_st

In [57]:
staff_df.columns = staff_df_col_names

In [58]:
staff_df.dropna(subset=[staff_df.columns[0]], inplace=True)

In [59]:
staff_df.reset_index(inplace=True)

In [60]:
#turn staff into a percentages

for i in staff_df.columns[2:]:
    col_name = i + "_as_percentage"
    staff_df[col_name] = (staff_df[i] / staff_df[staff_df.columns[1]]) * 100




In [61]:
staff_df = staff_df.round(2)

In [63]:
staff_df.head()

,UnitID,Grand total instructional_staff,Grand total men instructional_staff,Grand total women instructional_staff,American Indian or Alaska Native total instructional_staff,American Indian or Alaska Native men instructional_staff,American Indian or Alaska Native women instructional_staff,Asian total instructional_staff,Asian men instructional_staff,Asian women instructional_staff,Black or African American total instructional_staff,Black or African American men instructional_staff,Black or African American women instructional_staff,Hispanic or Latino total instructional_staff,Hispanic or Latino men instructional_staff,Hispanic or Latino women instructional_staff,Native Hawaiian or Other Pacific Islander total instructional_staff,Native Hawaiian or Other Pacific Islander men instructional_staff,Native Hawaiian or Other Pacific Islander women instructional_staff,White total instructional_staff,White men instructional_staff,White women instructional_staff,Two or more races total instructional_staff,Two or more races men instructional_staff,Two or more races women instructional_staff,Race/ethnicity unknown total instructional_staff,Race/ethnicity unknown men instructional_staff,Race/ethnicity unknown women instructional_staff,Nonresident alien total instructional_staff,Nonresident alien men instructional_staff,Nonresident alien women instructional_staff,Grand total men instructional_staff_as_percentage,Grand total women instructional_staff_as_percentage,American Indian or Alaska Native total instructional_staff_as_percentage,American Indian or Alaska Native men instructional_staff_as_percentage,American Indian or Alaska Native women instructional_staff_as_percentage,Asian total instructional_staff_as_percentage,Asian men instructional_staff_as_percentage,Asian women instructional_staff_as_percentage,Black or African American total instructional_staff_as_percentage,Black or African American men instructional_staff_as_percentage,Black or African American women instructional_staff_as_percentage,Hispanic or Latino total instructional_staff_as_percentage,Hispanic or Latino men instructional_staff_as_percentage,Hispanic or Latino women instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander total instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander men instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander women instructional_staff_as_percentage,White total instructional_staff_as_percentage,White men instructional_staff_as_percentage,White women instructional_staff_as_percentage,Two or more races total instructional_staff_as_percentage,Two or more races men instructional_staff_as_percentage,Two or more races women instructional_staff_as_percentage,Race/ethnicity unknown total instructional_staff_as_percentage,Race/ethnicity unknown men instructional_staff_as_percentage,Race/ethnicity unknown women instructional_staff_as_percentage,Nonresident alien total instructional_staff_as_percentage,Nonresident alien men instructional_staff_as_percentage,Nonresident alien women instructional_staff_as_percentage
0,180203,13.0,7.0,6.0,8.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.85,46.15,61.54,30.77,30.77,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,38.46,23.08,15.38,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,222178,265.0,154.0,111.0,1.0,1.0,0.0,3.0,2.0,1.0,14.0,7.0,7.0,13.0,8.0,5.0,0.0,0.0,0.0,228.0,131.0,97.0,3.0,2.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,58.11,41.89,0.38,0.38,0.00,1.13,0.75,0.38,5.28,2.64,2.64,4.91,3.02,1.89,0.00,0.00,0.00,86.04,49.43,36.60,1.13,0.75,0.38,0.00,0.00,0.00,1.13,1.13,0.00
2,138558,149.0,77.0,72.0,0.0,0.0,0.0,10.0,5.0,5.0,10.0,7.0,3.0,3.0,2.0,1.0,0.0,0.0,0.0,124.0,62.0,62.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,51.68,48.32,0.00,0.00,0.00,6.71,3.36,3.36,6.71,4.70,2.01,2.01,1.34,0.67,0.00,0.00,0.00,83.22,41.61,41.61,0.00,0.00,0.00,0.67,0.67,0.00,0.67,0.00,0.67
3,108232,219.0,124.0,95.0,0.

For our Study will use totals for simplicity. 

## Pell Grant Recipients


In [65]:
grant_df = numeric_df.loc[:,numeric_df.columns[32:34]].reset_index()

## Cohort Diversity

In [66]:
cohort_diversity_df = numeric_df.loc[:,numeric_df.columns[79:90]].reset_index()

## Age and Size of cohort

In [67]:
age_df = numeric_df.loc[:,numeric_df.columns[90:96]]

In [68]:
age_df["Incoming_Class_Grand_Total"] = age_df.iloc[:,0] + age_df.iloc[:,3]

In [69]:
for i in age_df.columns[:-1]:
    col_name = i + "_as_percentage"
    age_df[col_name] = age_df[i] / age_df["Incoming_Class_Grand_Total"]

In [70]:
age_df = age_df.round(2)

In [73]:
age_df.sample(10)

,Grand total (EF2013B_RV Undergraduate Age under 25 total),Total men (EF2013B_RV Undergraduate Age under 25 total),Total women (EF2013B_RV Undergraduate Age under 25 total),Grand total (EF2013B_RV Undergraduate Age 25 and over total),Total men (EF2013B_RV Undergraduate Age 25 and over total),Total women (EF2013B_RV Undergraduate Age 25 and over total),Incoming_Class_Grand_Total,Grand total (EF2013B_RV Undergraduate Age under 25 total)_as_percentage,Total men (EF2013B_RV Undergraduate Age under 25 total)_as_percentage,Total women (EF2013B_RV Undergraduate Age under 25 total)_as_percentage,Grand total (EF2013B_RV Undergraduate Age 25 and over total)_as_percentage,Total men (EF2013B_RV Undergraduate Age 25 and over total)_as_percentage,Total women (EF2013B_RV Undergraduate Age 25 and over total)_as_percentage
UnitID,,,,,,,,,,,,,
189413,431.0,108.0,323.0,619.0,135.0,484.0,1050.0,0.41,0.10,0.31,0.59,0.13,0.46
134343,3604.0,1455.0,2149.0,2667.0,995.0,1672.0,6271.0,0.57,0.23,0.34,0.43,0.16,0.27
138187,29245.0,13396.0,15849.0,12889.0,5420.0,7469.0,42134.0,0.69,0.32,0.38,0.31,0.13,0.18
197692,75.0,75.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228705,6267.0,3088.0,3179.0,3145.0,1208.0,1937.0,9412.0,0.67,0.33,0.34,0.33,0.13,0.21
440794,79.0,41.0,38.0,339.0,113.0,226.0,418.0,0.19,0.10,0.09,0.81,0.27,0.54
202480,7716.0,3984.0,3732.0,257.0,137.0,120.0,7973.0,0.97,0.50,0.47,0.03,0.02,0.02
431983,126.0,126.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213543,4901.0,2753.0,2148.0,28.0,19.0,9.0,4929.0,0.99,0.56,0.44,0.01,0.00,0.00


In [74]:
age_df.isna().sum()

Grand total (EF2013B_RV  Undergraduate  Age under 25 total)                      66
Total men (EF2013B_RV  Undergraduate  Age under 25 total)                        66
Total women (EF2013B_RV  Undergraduate  Age under 25 total)                      66
Grand total (EF2013B_RV  Undergraduate  Age 25 and over total)                   99
Total men (EF2013B_RV  Undergraduate  Age 25 and over total)                     99
Total women (EF2013B_RV  Undergraduate  Age 25 and over total)                   99
Incoming_Class_Grand_Total                                                      101
Grand total (EF2013B_RV  Undergraduate  Age under 25 total)_as_percentage       101
Total men (EF2013B_RV  Undergraduate  Age under 25 total)_as_percentage         101
Total women (EF2013B_RV  Undergraduate  Age under 25 total)_as_percentage       101
Grand total (EF2013B_RV  Undergraduate  Age 25 and over total)_as_percentage    101
Total men (EF2013B_RV  Undergraduate  Age 25 and over total)_as_percentage  

## Price 

In [75]:
price_df = numeric_df.loc[:,numeric_df.columns[98]].reset_index()

In [76]:
price_df

,UnitID,Total price for out-of-state students living on campus 2012-13 (DRVIC2012_RV)
0,180203,NaN
1,222178,39900.0
2,138558,22684.0
3,172866,NaN
4,108232,36884.0
...,...,...
2310,217059,28820.0
2311,141361,34093.0
2312,206695,20580.0
2313,204255,NaN


## Student Faculty Ratio 

In [77]:
student_faculty = numeric_df.loc[:,numeric_df.columns[105]].reset_index()

In [78]:
student_faculty.head()

,UnitID,Student-to-faculty ratio (EF2019D)
0,180203,10.0
1,222178,15.0
2,138558,17.0
3,172866,9.0
4,108232,14.0


## SAT Scores

In [79]:
test_scores = numeric_df.loc[:,numeric_df.columns[106:]].reset_index()

In [80]:
test_scores.drop(test_scores.columns[5:], axis = 1, inplace=True)

In [81]:
test_scores.drop(test_scores.columns[3], axis=1, inplace=True)

# Merging all cleaned dataframes on UnitId

In [103]:
cleaned_df_list = [
    finances_df, staff_df, grant_df, 
    cohort_diversity_df, age_df, price_df, 
    student_faculty, test_scores,
    categorical_df_no_city, dependent_df
]

In [104]:
pre_dummies_df = reduce(lambda x, y: pd.merge(x, y, on='UnitID', how="outer"), cleaned_df_list)

In [105]:
pre_dummies_df.to_csv("data/pre_dummies.csv")

In [109]:
pre_dummies_df.iloc[:,-5].unique()

array([ 33.,  29.,  38.,  47.,  22.,  57.,  37.,  nan,  45.,  36.,  74.,
        26.,  64.,   5.,  59.,  54.,  51.,  56.,  41.,  14.,  81.,  40.,
        39.,  10.,  44.,  27.,  68.,  49.,  83.,  31.,  52.,  17.,  65.,
        63.,  84.,  73.,  62.,  48.,  67.,  72.,  77.,  55.,  23., 100.,
        75.,  43.,  42.,  50.,  25.,  80.,  20.,  34.,  19.,  85.,  66.,
        30.,  12.,  13.,  24.,  71.,  15.,   0.,  61.,  58.,  46.,  53.,
        35.,  70.,  90.,  69.,  18.,  28.,   7.,  21.,  82.,   9.,  32.,
        60.,  87.,  76.,  78.,  88.,  91.,  86.,  92.,  89.,  94.,  95.,
        79.,   8.,  93.,  11.,   6.,  97.,  96.,  98.,   4.,  16.])

In [110]:
pre_imputed = pd.get_dummies(pre_dummies_df, drop_first=True)

In [117]:
graduation_string = pre_imputed.columns[119]

In [113]:
pre_imputed.columns[119:150]

Index(['Graduation rate  total cohort (DRVGR2019)',
       'Graduation rate  men (DRVGR2019)',
       'Graduation rate  women (DRVGR2019)',
       'Graduation rate  American Indian or Alaska Native (DRVGR2019)',
       'Graduation rate  Asian/Native Hawaiian/Other Pacific Islander (DRVGR2019)',
       'Graduation rate  Asian (DRVGR2019)',
       'Graduation rate  Native Hawaiian or Other Pacific Islander (DRVGR2019)',
       'Graduation rate  Black  non-Hispanic (DRVGR2019)',
       'Graduation rate  Hispanic (DRVGR2019)',
       'Graduation rate  White  non-Hispanic (DRVGR2019)',
       'Graduation rate  two or more races (DRVGR2019)',
       'Graduation rate  Race/ethnicity unknown (DRVGR2019)',
       'Graduation rate  Nonresident alien (DRVGR2019)',
       'Transfer-out rate  total cohort (DRVGR2019)',
       'Carnegie Classification 2018: Basic (HD2018)_Associate's Colleges: High Transfer-Mixed Traditional/Nontraditional',
       'Carnegie Classification 2018: Basic (HD2018)_Assoc

# Finding The Best Imputation Technique

In [118]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.pipeline import make_pipeline, Pipeline

In [156]:
x = pre_imputed.drop(graduation_string, axis=1)
y = pre_imputed[graduation_string]

In [153]:
dependent_df.columns[:]

Index(['UnitID', 'Graduation rate  total cohort (DRVGR2019)',
       'Graduation rate  men (DRVGR2019)',
       'Graduation rate  women (DRVGR2019)',
       'Graduation rate  American Indian or Alaska Native (DRVGR2019)',
       'Graduation rate  Asian/Native Hawaiian/Other Pacific Islander (DRVGR2019)',
       'Graduation rate  Asian (DRVGR2019)',
       'Graduation rate  Native Hawaiian or Other Pacific Islander (DRVGR2019)',
       'Graduation rate  Black  non-Hispanic (DRVGR2019)',
       'Graduation rate  Hispanic (DRVGR2019)',
       'Graduation rate  White  non-Hispanic (DRVGR2019)',
       'Graduation rate  two or more races (DRVGR2019)',
       'Graduation rate  Race/ethnicity unknown (DRVGR2019)',
       'Graduation rate  Nonresident alien (DRVGR2019)',
       'Transfer-out rate  total cohort (DRVGR2019)'],
      dtype='object')

In [157]:
x.drop(dependent_df.columns[2:], axis=1, inplace=True)

In [224]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.12, random_state=42)

### Simple Imputation

In [225]:
simple_pipeline = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('classifier', Lasso(alpha=0.1))
])

In [226]:
simple_pipeline.fit(x_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('classifier', Lasso(alpha=0.1))])

In [227]:
# scaler = StandardScaler()
# x_test_scaled = scaler.fit_transform(x_test)

In [228]:
y_pred_simple = simple_pipeline.predict(x_test)

In [229]:
mean_squared_error(y_pred_simple, y_test)

212.72563828611266

### Iterative Imputation

In [190]:
iterative_pipeline = Pipeline([
    ('imputer', IterativeImputer()),
    ('scaler', StandardScaler()),
    ('classifier', Lasso(alpha=0.1))
])

In [191]:
iterative_pipeline.fit(x_train, y_train)

/Users/robertcampbell/opt/anaconda3/lib/python3.8/site-packages/sklearn/impute/_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


Pipeline(steps=[('imputer', IterativeImputer()), ('scaler', StandardScaler()),
                ('classifier', Lasso(alpha=0.1))])

In [192]:
y_pred_itr = iterative_pipeline.predict(x_test)

In [193]:
mean_squared_error(y_pred_itr, y_test)

275.2995110814378

### KNN Imputation

In [230]:
KNN_pipeline = Pipeline([
    ('imputer', KNNImputer()),
    ('scaler', StandardScaler()),
    ('classifier', Lasso(alpha=0.1))
])

In [231]:
KNN_pipeline.fit(x_train, y_train)

Pipeline(steps=[('imputer', KNNImputer()), ('scaler', StandardScaler()),
                ('classifier', Lasso(alpha=0.1))])

In [232]:
y_pred_knn = KNN_pipeline.predict(x_test)

In [233]:
mean_squared_error(y_test, y_pred_knn)

212.72563828611266

In [234]:
# KNN_pipeline.named_steps["classifier"].score(x_test, y_test)
explained_variance_score(y_test, y_pred_knn)

0.5476564198975499

### KNN imputation gives us the best model performence so lets create our final cleaned dataframe with KNN

In [235]:
x.isna().sum()

UnitID                                       0
Core_Revenues                                0
Tuition_And_Fees                             0
Government_Grants                            0
Private_Gifts                                0
                                            ..
State abbreviation (HD2018)_Washington       0
State abbreviation (HD2018)_West Virginia    0
State abbreviation (HD2018)_Wisconsin        0
State abbreviation (HD2018)_Wyoming          0
State abbreviation (HD2018)_isMissing        0
Length: 254, dtype: int64

In [236]:
KNN_imputer = KNNImputer()

In [237]:
x.loc[:,:] = KNN_imputer.fit_transform(x)

In [238]:
x.isna().sum()

UnitID                                       0
Core_Revenues                                0
Tuition_And_Fees                             0
Government_Grants                            0
Private_Gifts                                0
                                            ..
State abbreviation (HD2018)_Washington       0
State abbreviation (HD2018)_West Virginia    0
State abbreviation (HD2018)_Wisconsin        0
State abbreviation (HD2018)_Wyoming          0
State abbreviation (HD2018)_isMissing        0
Length: 254, dtype: int64

In [239]:
x[graduation_string] = y

### We will be studying total graduation rates along with African American Graduation Rates

In [240]:
black_grad_rates_df = pd.DataFrame(dependent_df[["UnitID", dependent_df.columns[8]]])
black_grad_rates_df.head()

,UnitID,Graduation rate Black non-Hispanic (DRVGR2019)
0,180203,NaN
1,222178,53.0
2,138558,12.0
3,172866,0.0
4,108232,20.0


In [241]:
cleaned_df = x.merge(black_grad_rates_df, on="UnitID", how="outer")

In [242]:
cleaned_df.set_index("UnitID", inplace=True)

In [243]:
cleaned_df.to_csv("data/cleaned_df.csv")